<a href="https://colab.research.google.com/github/Meenakumari106/AI_Image_generating/blob/main/NextWordPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving blue_castle.txt to blue_castle.txt


In [ ]:
file = open("blue_castle.txt", "r", encoding = "utf8" )

#store file in list
lines = []
for i in file:
     lines.append(i)

#Convert list to string
data = ""
for i in lines:
    data = '  '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]


'The Project Gutenberg eBook of The Blue Castle, by Lucy Maud Montgomery This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before usi'

In [ ]:
len(data)

402906

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl','wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 112, 97, 587, 4, 1, 94, 147, 58, 2383, 2384, 1818, 51, 587, 42]

In [ ]:
len(sequence_data)

72052

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8413


In [ ]:
sequences = []

for i in range (3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print("The Length if sequences are:" , len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length if sequences are: 72049


array([[   1,  112,   97,  587],
       [ 112,   97,  587,    4],
       [  97,  587,    4,    1],
       [ 587,    4,    1,   94],
       [   4,    1,   94,  147],
       [   1,   94,  147,   58],
       [  94,  147,   58, 2383],
       [ 147,   58, 2383, 2384],
       [  58, 2383, 2384, 1818],
       [2383, 2384, 1818,   51]])

In [ ]:
X = []
y = []

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])

X = np.array(X)
y = np.array(y)

In [ ]:
print("Data:", X[:10])
print("Response:", y[:10])

Data: [[   1  112   97]
 [ 112   97  587]
 [  97  587    4]
 [ 587    4    1]
 [   4    1   94]
 [   1   94  147]
 [  94  147   58]
 [ 147   58 2383]
 [  58 2383 2384]
 [2383 2384 1818]]
Response: [ 587    4    1   94  147   58 2383 2384 1818   51]


In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation= "relu"))
model.add(Dense(vocab_size, activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

checkpoint = ModelCheckpoint("next_word.h5", monitor='loss', verbose=1, save_best_only=True)

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])


Epoch 1/20
1124/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 7.0311
Epoch 1: loss improved from inf to 6.76203, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 39s 27ms/step - loss: 7.0304
Epoch 2/20
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 6.2436
Epoch 2: loss improved from 6.76203 to 6.20945, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 38s 26ms/step - loss: 6.2435
Epoch 3/20
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 5.8615
Epoch 3: loss improved from 6.20945 to 5.83714, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 29s 25ms/step - loss: 5.8614
Epoch 4/20
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 5.5439
Epoch 4: loss improved from 5.83714 to 5.54659, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 47s 31ms/step - loss: 5.5439
Epoch 5/20
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 5.2864
Epoch 5: loss improved from 5.54659 to 5.28929, saving model to next_word.h5


...

1126/1126 ━━━━━━━━━━━━━━━━━━━━ 30s 27ms/step - loss: 2.4006
Epoch 17/20
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.1760
Epoch 17: loss improved from 2.49492 to 2.27657, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 30s 27ms/step - loss: 2.1762
Epoch 18/20
1124/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.9717
Epoch 18: loss improved from 2.27657 to 2.07252, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 30s 27ms/step - loss: 1.9720
Epoch 19/20
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.7804
Epoch 19: loss improved from 2.07252 to 1.87453, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 30s 27ms/step - loss: 1.7806
Epoch 20/20
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.5875
Epoch 20: loss improved from 1.87453 to 1.67984, saving model to next_word.h5


1126/1126 ━━━━━━━━━━━━━━━━━━━━ 30s 26ms/step - loss: 1.5877


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#Load the model and tokenizer
model = load_model('next_word.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predict_word = ""

  for key, value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line:")

  if text == "0":
    print("Execution completed....")
    break

  else:
    try:
      text = text.split(" ")
      text = text[-3:]
      print(text)

      Predict_Next_Words(model, tokenizer, text)


    except Exception as e:
       print("Error occured: ",e)
       continue